In [33]:
from google.colab import files
files.upload()
!ls


data.csv  sample_data  topsis.py


In [36]:
%%writefile topsis.py
import sys
import pandas as pd
import numpy as np


def run_topsis(input_file, weight_str, impact_str, output_file):

    try:
        df = pd.read_csv(input_file)
    except FileNotFoundError:
        print("Error: Input file not found.")
        sys.exit(1)

    if df.shape[1] < 3:
        print("Error: File must contain at least 3 columns.")
        sys.exit(1)

    try:
        matrix = df.iloc[:, 1:].astype(float).values
    except:
        print("Error: Criteria values must be numeric.")
        sys.exit(1)

    criteria_count = matrix.shape[1]

    weights = weight_str.split(',')
    if len(weights) != criteria_count:
        print("Error: Number of weights is incorrect.")
        sys.exit(1)

    weights = np.array(weights, dtype=float)

    impacts = impact_str.split(',')
    if len(impacts) != criteria_count:
        print("Error: Number of impacts is incorrect.")
        sys.exit(1)

    for sign in impacts:
        if sign not in ['+', '-']:
            print("Error: Impacts must be '+' or '-'.")
            sys.exit(1)

    normalized = matrix / np.sqrt((matrix ** 2).sum(axis=0))
    weighted = normalized * weights

    best = []
    worst = []

    for i in range(criteria_count):
        column = weighted[:, i]
        if impacts[i] == '+':
            best.append(column.max())
            worst.append(column.min())
        else:
            best.append(column.min())
            worst.append(column.max())

    best = np.array(best)
    worst = np.array(worst)

    dist_best = np.sqrt(((weighted - best) ** 2).sum(axis=1))
    dist_worst = np.sqrt(((weighted - worst) ** 2).sum(axis=1))

    score = dist_worst / (dist_best + dist_worst)

    df["Topsis Score"] = score
    df["Rank"] = df["Topsis Score"].rank(ascending=False).astype(int)

    df.to_csv(output_file, index=False)
    print("TOPSIS completed successfully.")


if __name__ == "__main__":

    if len(sys.argv) != 5:
        print("Usage: python topsis.py <InputFile> <Weights> <Impacts> <OutputFile>")
        sys.exit(1)

    run_topsis(sys.argv[1], sys.argv[2], sys.argv[3], sys.argv[4])


Overwriting topsis.py


In [37]:
!ls


data.csv  sample_data  topsis.py


In [38]:
!python topsis.py data.csv "1,1,1,1" "-,+,+,+" result.csv


TOPSIS completed successfully.


In [39]:
import pandas as pd
pd.read_csv("result.csv")


,Model,Price,Mileage,Comfort,Safety,Topsis Score,Rank
0,A,250000,15,7,8,0.643383,1
1,B,300000,12,8,9,0.571164,2
2,C,280000,14,6,7,0.289393,3
